In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from gensim.models import KeyedVectors
from util import load_metric, read_util
import json

In [ ]:
#import gensim

In [2]:
dwt_roots = read_util(Path("../data/utils/dwts.txt"))

In [3]:
dwt_roots

['förortsgäng',
 'återvandr',
 'berika',
 'kulturberika',
 'ordning_och_reda_i_flyktingpolitiken',
 'globalist',
 'hjälpa_på_plats',
 'självständig_utrikespolitik']

In [4]:
models_at = Path("/home/max/Results/fb_pol-yearly-rad3/models")
vocabs_at = Path("/home/max/Corpora/flashback-pol-time/yearly/fb-pt-radical3-v0/vocab")

In [5]:
def neighbors(mod_path, voc_path, roots, restriction = ("N1", "N2", "V1", "V2", "A1", "P1"), k=10):
    
    lst = list()
    
    for model in sorted(os.listdir(mod_path)):
        
        if not model.endswith(".w2v"):
            continue
        wv = KeyedVectors.load_word2vec_format(mod_path / model)
        counts = load_metric(voc_path / model.replace(".w2v", ".txt"))
        words = [word for word in wv.index_to_key if any(root in word for root in roots)]
        if restriction != None:
            words = [word for word in words if word.startswith(restriction)]
        
        year = model.replace(".w2v", "")
        
        for word in words:
            for neighbor, score in wv.most_similar(positive = [word], topn = k):
                count = int(counts[neighbor])
                lst.append({
                    "Word": word,
                    "Year": year,
                    "Neighbor": neighbor,
                    "Score": score,
                    "Count": count
                })
                
    return lst

In [6]:
data = neighbors(models_at, vocabs_at, dwt_roots, k=15)

In [ ]:
sorted(to_json.keys())

In [ ]:
to_json

In [ ]:
with open(Path("../../dw_results/neighbors.json"), "w") as f:
    f.write(json.dumps(data, indent=4))  

In [ ]:
df = pd.DataFrame(data)
df.to_csv("../../dw_results/neighbors.csv", sep="\t")